In [1]:
%load_ext autoreload
%autoreload 2

# Import library
import pandas as pd
import numpy as np
import re
import os
from tqdm import tqdm
from IPython.display import display
from glob import glob
import cv2


# Import dependency
from src.preprocessing.generate_metadata import discover_wsi
from src.preprocessing.xml_to_mask import get_mask
from src.preprocessing.annotation_utils import resolve_annotation_path
from src.preprocessing.extract_patches import extract_patches
from src.preprocessing.load_wsi import load_wsi


In [2]:
# Configuration
BASE_DIR = 'data'
SOURCES = [
    'Yale_HER2_cohort',
    'Yale_trastuzumab_response_cohort',
    'TCGA_BRCA_Filtered'
]
OUTPUT_CSV = 'outputs/index/wsi_index.csv'

In [3]:
log_dir = 'outputs/preprocessing/logs'
os.makedirs(log_dir, exist_ok=True)
log_path = os.path.join(log_dir, 'preprocessing.log')

def log(msg):
    print(msg)
    with open(log_path, 'a') as f:
        f.write(msg + '\n')

In [4]:
csv_path = discover_wsi(
    base_dir=BASE_DIR, 
    sources=SOURCES, 
    output_path=OUTPUT_CSV
)

# Load and display the results
df = pd.read_csv(csv_path)
display(df.head(50))

Processing sources: 100%|██████████| 3/3 [00:00<00:00, 59.80it/s]
                                                         

wsi_path         slide_id  \
0   data/Yale_HER2_cohort/SVS/Her2Neg_Case_01.svs  Her2Neg_Case_01   
1   data/Yale_HER2_cohort/SVS/Her2Neg_Case_02.svs  Her2Neg_Case_02   
2   data/Yale_HER2_cohort/SVS/Her2Neg_Case_03.svs  Her2Neg_Case_03   
3   data/Yale_HER2_cohort/SVS/Her2Neg_Case_04.svs  Her2Neg_Case_04   
4   data/Yale_HER2_cohort/SVS/Her2Neg_Case_05.svs  Her2Neg_Case_05   
5   data/Yale_HER2_cohort/SVS/Her2Neg_Case_06.svs  Her2Neg_Case_06   
6   data/Yale_HER2_cohort/SVS/Her2Neg_Case_07.svs  Her2Neg_Case_07   
7   data/Yale_HER2_cohort/SVS/Her2Neg_Case_08.svs  Her2Neg_Case_08   
8   data/Yale_HER2_cohort/SVS/Her2Neg_Case_09.svs  Her2Neg_Case_09   
9   data/Yale_HER2_cohort/SVS/Her2Neg_Case_10.svs  Her2Neg_Case_10   
10  data/Yale_HER2_cohort/SVS/Her2Neg_Case_11.svs  Her2Neg_Case_11   
11  data/Yale_HER2_cohort/SVS/Her2Neg_Case_12.svs  Her2Neg_Case_12   
12  data/Yale_HER2_cohort/SVS/Her2Neg_Case_13.svs  Her2Neg_Case_13   
13  data/Yale_HER2_cohort/SVS/Her2Neg_Case_14.svs  Her2Neg_Case_14   
14  data/Yale_HER2_cohort/SVS/Her2Neg_Case_15.svs  Her2Neg_Case_15   
15  data/Yale_HER2_cohort/SVS/Her2Neg_Case_16.svs  Her2Neg_Case_16   
16  data/Yale_HER2_cohort/SVS/Her2Neg_Case_17.svs  Her2Neg_Case_17   
17  data/Yale_HER2_cohort/SVS/Her2Neg_Case_18.svs  Her2Neg_Case_18   
18  data/Yale_HER2_cohort/SVS/Her2Neg_Case_19.svs  Her2Neg_Case_19   
19  data/Yale_HER2_cohort/SVS/Her2Neg_Case_20.svs  Her2Neg_Case_20   
20  data/Yale_HER2_cohort/SVS/Her2Neg_Case_21.svs  Her2Neg_Case_21   
21  data/Yale_HER2_cohort/SVS/Her2Neg_Case_22.svs  Her2Neg_Case_22   
22  data/Yale_HER2_cohort/SVS/Her2Neg_Case_23.svs  Her2Neg_Case_23   
23  data/Yale_HER2_cohort/SVS/Her2Neg_Case_24.svs  Her2Neg_Case_24   
24  data/Yale_HER2_cohort/SVS/Her2Neg_Case_25.svs  Her2Neg_Case_25   
25  data/Yale_HER2_cohort/SVS/Her2Neg_Case_26.svs  Her2Neg_Case_26   
26  data/Yale_HER2_cohort/SVS/Her2Neg_Case_27.svs  Her2Neg_Case_27   
27  data/Yale_HER2_cohort/SVS/Her2Neg_Case_28.svs  Her2Neg_Case_28   
28  data/Yale_HER2_cohort/SVS/Her2Neg_Case_29.svs  Her2Neg_Case_29   
29  data/Yale_HER2_cohort/SVS/Her2Neg_Case_30.svs  Her2Neg_Case_30   
30  data/Yale_HER2_cohort/SVS/Her2Neg_Case_31.svs  Her2Neg_Case_31   
31  data/Yale_HER2_cohort/SVS/Her2Neg_Case_32.svs  Her2Neg_Case_32   
32  data/Yale_HER2_cohort/SVS/Her2Neg_Case_33.svs  Her2Neg_Case_33   
33  data/Yale_HER2_cohort/SVS/Her2Neg_Case_34.svs  Her2Neg_Case_34   
34  data/Yale_HER2_cohort/SVS/Her2Neg_Case_35.svs  Her2Neg_Case_35   
35  data/Yale_HER2_cohort/SVS/Her2Neg_Case_36.svs  Her2Neg_Case_36   
36  data/Yale_HER2_cohort/SVS/Her2Neg_Case_37.svs  Her2Neg_Case_37   
37  data/Yale_HER2_cohort/SVS/Her2Neg_Case_38.svs  Her2Neg_Case_38   
38  data/Yale_HER2_cohort/SVS/Her2Neg_Case_39.svs  Her2Neg_Case_39   
39  data/Yale_HER2_cohort/SVS/Her2Neg_Case_40.svs  Her2Neg_Case_40   
40  data/Yale_HER2_cohort/SVS/Her2Neg_Case_41.svs  Her2Neg_Case_41   
41  data/Yale_HER2_cohort/SVS/Her2Neg_Case_42.svs  Her2Neg_Case_42   
42  data/Yale_HER2_cohort/SVS/Her2Neg_Case_43.svs  Her2Neg_Case_43   
43  data/Yale_HER2_cohort/SVS/Her2Neg_Case_44.svs  Her2Neg_Case_44   
44  data/Yale_HER2_cohort/SVS/Her2Neg_Case_45.svs  Her2Neg_Case_45   
45  data/Yale_HER2_cohort/SVS/Her2Neg_Case_46.svs  Her2Neg_Case_46   
46  data/Yale_HER2_cohort/SVS/Her2Neg_Case_47.svs  Her2Neg_Case_47   
47  data/Yale_HER2_cohort/SVS/Her2Neg_Case_48.svs  Her2Neg_Case_48   
48  data/Yale_HER2_cohort/SVS/Her2Neg_Case_49.svs  Her2Neg_Case_49   
49  data/Yale_HER2_cohort/SVS/Her2Neg_Case_50.svs  Her2Neg_Case_50   

             slide_name      annotation_name  \
0   Her2Neg_Case_01.svs  Her2Neg_Case_01.xml   
1   Her2Neg_Case_02.svs  Her2Neg_Case_02.xml   
2   Her2Neg_Case_03.svs  Her2Neg_Case_03.xml   
3   Her2Neg_Case_04.svs  Her2Neg_Case_04.xml   
4   Her2Neg_Case_05.svs  Her2Neg_Case_05.xml   
5   Her2Neg_Case_06.svs  Her2Neg_Case_06.xml   
6   Her2Neg_Case_07.svs  Her2Neg_Case_07.xml   
7   Her2Neg_Case_08.svs  Her2Neg_Case_08.xml   
8   Her2Neg_Case_09.svs  Her2Neg_Case_

In [10]:
for idx, row in tqdm(df.iterrows(), total=len(df), desc='Processing slides'):
    wsi_path = row['wsi_path']
    # Resolve annotation path using helper (handles pandas NA, relative paths, and glob fallback)
    annotation_path = resolve_annotation_path(row.get('annotation_path', None), wsi_path, base_dir=BASE_DIR)
    if not annotation_path:
        log(f"Skipping slide without annotation: {wsi_path}")
        continue

    log(f"Processing slide: {wsi_path} with annotation: {annotation_path}")
    try:
        mask = get_mask(annotation_path, wsi_path)
    except Exception as e:
        log(f"Failed to generate mask for {wsi_path}: {e}")
        continue
    if mask is None:
        log(f"No mask generated for {wsi_path}")
        continue

    # At this point mask should be a 2D uint8 array (0 or 255)
    log(f'Mask shape: {mask.shape}')

    # Load WSI using the wrapper which prefers CuCIM when available
    try:
        wsi_slide = load_wsi(wsi_path)
    except Exception as e:
        log(f"Failed to load WSI ({wsi_path}): {e}")
        continue
    if wsi_slide is None:
        log(f"Failed to load WSI: {wsi_path}")
        continue

    # Log which backend the loader selected (cucim or openslide)
    backend = getattr(wsi_slide, 'backend', None)
    log(f'Loaded WSI backend: {backend}')

    # Extract patches (extract_patches can optionally save patches to disk)
    slide_base = os.path.splitext(os.path.basename(wsi_path))[0]
    out_dir_patches = os.path.join('outputs', 'patches', slide_base)
    try:
        patches = extract_patches(
            wsi_slide,
            mask=mask,
            size=512,
            stride=512,
            save_dir=out_dir_patches,
            save_prefix=slide_base,
            save_format='png'
        )
    except Exception as e:
        log(f"Failed to extract patches for {wsi_path}: {e}")
        continue

    saved = sum(1 for p in patches if p.get('path'))
    log(f'Extracted {len(patches)} patches from {wsi_path}; saved {saved} to {out_dir_patches}')

Processing slides:   0%|          | 0/471 [00:00<?, ?it/s]

Processing slide: data/Yale_HER2_cohort/SVS/Her2Neg_Case_01.svs with annotation: /media/thanakornbuath/Phone SSD/her2-attention-classifier/data/Yale_HER2_cohort/Annotations/Her2Neg_Case_01.xml
Mask shape: (45426, 41888)
Loaded WSI backend: cucim
Mask shape: (45426, 41888)
Loaded WSI backend: cucim


Processing slides:   0%|          | 1/471 [00:07<55:48,  7.13s/it]

Extracted 155 patches from data/Yale_HER2_cohort/SVS/Her2Neg_Case_01.svs; saved 155 to outputs/patches/Her2Neg_Case_01
Processing slide: data/Yale_HER2_cohort/SVS/Her2Neg_Case_02.svs with annotation: /media/thanakornbuath/Phone SSD/her2-attention-classifier/data/Yale_HER2_cohort/Annotations/Her2Neg_Case_02.xml
Mask shape: (28692, 31416)
Loaded WSI backend: cucim


Processing slides:   0%|          | 2/471 [00:11<42:18,  5.41s/it]

...

Extracted 258 patches from data/Yale_HER2_cohort/SVS/Her2Neg_Case_16.svs; saved 258 to outputs/patches/Her2Neg_Case_16
Processing slide: data/Yale_HER2_cohort/SVS/Her2Neg_Case_17.svs with annotation: /media/thanakornbuath/Phone SSD/her2-attention-classifier/data/Yale_HER2_cohort/Annotations/Her2Neg_Case_17.xml
Mask shape: (16635, 37128)
Loaded WSI backend: cucim


Processing slides:   4%|▎         | 17/471 [01:46<1:04:16,  8.49s/it]

Extracted 61 patches from data/Yale_HER2_cohort/SVS/Her2Neg_Case_17.svs; saved 61 to outputs/patches/Her2Neg_Case_17
Processing slide: data/Yale_HER2_cohort/SVS/Her2Neg_Case_18.svs with annotation: /media/thanakornbuath/Phone SSD/her2-attention-classifier/data/Yale_HER2_cohort/Annotations/Her2Neg_Case_18.xml
Mask shape: (40669, 54264)
Loaded WSI backend: cucim
Mask shape: (40669, 54264)
Loaded WSI backend: cucim


Processing slides:   4%|▎         | 17/471 [01:52<50:01,  6.61s/it]  



KeyboardInterrupt: 